# Parser Unimed Odonto
Este notebook extrai procedimentos de um relatório Unimed Odonto em PDF e exporta para Excel (.xls).

In [ ]:
# 📌 PASSO 1 - Instala as dependências necessárias
!pip install pdfplumber xlwt

In [ ]:
# 📁 PASSO 2 - Faz upload do PDF manualmente (ou use do Google Drive)
from google.colab import files
uploaded = files.upload()

import os
for fname in uploaded.keys():
    pdf_path = os.path.abspath(fname)
print("PDF carregado:", pdf_path)

In [ ]:
# 🔎 PASSO 3 - Função principal para extração dos dados
import pdfplumber
import pandas as pd
import re

def extrair_dados_unimed(pdf_path):
    dados = []
    gto_atual = ""
    codigo_paciente = ""
    nome_paciente = ""

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            linhas = page.extract_text().split('\n')
            for linha in linhas:
                match_gto = re.match(r"GTO:(\d+)\s+CÓDIGO E NOME DO BENEFICIÁRIO:\s+(\d+)\s+-\s+(.+)", linha)
                if match_gto:
                    gto_atual = match_gto.group(1).strip()
                    codigo_paciente = match_gto.group(2).strip()
                    nome_paciente = match_gto.group(3).strip()
                    continue

                match_proc = re.match(
                    r"(\d{8}) (.+?)\s+(\S+)\s+(Pago|Glosado|Deferido|Indeferido)\s+([\d,]+)\s+([\d,]+)\s+([\d,]+)\s+(\d{2}/\d{2}/\d{4})",
                    linha
                )

                if match_proc:
                    cod_proc = match_proc.group(1)
                    desc_proc = match_proc.group(2).strip()
                    detalhe_face = match_proc.group(3).strip()
                    status = match_proc.group(4)
                    valor_proc = match_proc.group(5).replace(",", ".")
                    valor_glosa = match_proc.group(6).replace(",", ".")
                    valor_final = match_proc.group(7).replace(",", ".")
                    data = match_proc.group(8)

                    if detalhe_face.isdigit():
                        detalhe = detalhe_face
                        face = ""
                    elif detalhe_face.isalpha():
                        detalhe = ""
                        face = detalhe_face
                    else:
                        detalhe = detalhe_face[:-2]
                        face = detalhe_face[-2:]

                    dados.append({
                        "Data do atendimento": data,
                        "Código do paciente": codigo_paciente,
                        "Nome do paciente": nome_paciente,
                        "Descrição do procedimento": desc_proc,
                        "Detalhe": detalhe,
                        "Face": face,
                        "Número da GTO": gto_atual,
                        "Status do procedimento": status,
                        "Valor do procedimento (R$)": valor_proc,
                        "Valor glosado (R$)": valor_glosa,
                        "Valor final (R$)": valor_final
                    })

    return pd.DataFrame(dados)

df = extrair_dados_unimed(pdf_path)
df.head()


In [ ]:

# 💾 PASSO 4 - Exporta o DataFrame para Excel
df.to_excel("procedimentos_unimed.xlsx", index=False, engine='openpyxl')
files.download("procedimentos_unimed.xlsx")
